In [1]:
import pandas as pd
from tqdm import tqdm
import numpy as np
import cv2
import os
from datetime import timedelta
import sys
from pathlib import Path 

In [2]:
info = {}
with open('train/разметка.txt', 'r') as f:
    for line in f:
        if 'mp4' in line:
            file_name = line.strip()
            info[file_name] = []
        else:
            if line.strip() != '':
                info[file_name].append(line.strip())


In [3]:
info

{'0000000_00000020240221082923_0001_IMP (1).mp4': ['01:43',
  '06:10',
  '10:49',
  '10:56'],
 '0000000_00000020240221084423_0002_IMP.mp4': ['07:05', '08:10'],
 '0000000_00000020240221085923_0003_IMP.mp4': ['09:15', '13:38', '13:48'],
 '0000000_00000020240508114229_0002.mp4': ['02:28', '05:33', '07:04'],
 '0000000_00000020240508132920_0006.mp4': ['00:47', '01:53'],
 '2_5395803543229709215.mp4': ['00:01',
  '02:01',
  '02:18',
  '02:47',
  '03:12',
  '11:47',
  '12:43'],
 '2_5395803543229709261.mp4': ['01:43'],
 '2_5395803543229709728.mp4': ['01:20'],
 '2_5395803543229709828.mp4': ['04:39', '05:10'],
 '27e2fbc6-8579-4f48-852d-a42fd49522a2.mp4': ['-'],
 'VID_20240304_095112.mp4': ['00:21'],
 'VID_20240304_095227.mp4': ['-'],
 'VID_20240304_095302.mp4': ['-'],
 'VID_20240304_095352.mp4': ['00:07'],
 'VID_20240304_095423.mp4': ['00:03'],
 'VID_20240305_130307.mp4': ['-'],
 'VID_20240305_130357.mp4': ['-'],
 'VID_20240305_130642.mp4': ['-'],
 'VID_20240305_130713.mp4': ['-'],
 'VID_20240311

In [4]:
files_wo_markup = []
for key, val in info.items():
    if val[0] == '-':
        files_wo_markup.append(key)

In [5]:
len(files_wo_markup)

22

In [6]:
len(info.keys())

38

In [7]:
def format_timedelta(td):
    """Служебная функция для классного форматирования объектов timedelta (например, 00:00:20.05)
    исключая микросекунды и сохраняя миллисекунды"""
    result = str(td)
    try:
        result, ms = result.split(".")
    except ValueError:
        return (result + ".00").replace(":", "-")
    ms = int(ms)
    ms = round(ms / 1e4)
    return f"{result}.{ms:02}".replace(":", "-")




In [8]:
def get_saving_frames_durations(cap, saving_fps, df, delta = 5):
    """Функция, которая возвращает список длительностей, в которые следует сохранять кадры.
    df - инфо по нарушениям 
    delta - сколько секунд отступаем до нарушения
    """

    # получаем продолжительность клипа, разделив количество кадров на количество кадров в секунду
    clip_duration = cap.get(cv2.CAP_PROP_FRAME_COUNT) / cap.get(cv2.CAP_PROP_FPS)
        
    flags = [] # метки флага нарушений
    number_of_violent = [] # номер нарушения в кадре
    s = [] 
    end_old = -1
    for idx, row in df.iterrows():

        m_start = row.start.minute
        s_start = row.start.second

        m_end = row.end.minute
        s_end = row.end.second

         # преобразуем время нарушений в секунды    
        start = m_start*60 + s_start
        end = m_end*60 + s_end
        
        # определяем время начала нарушения
        if start-delta < end_old:
            start_new = start
        else: 
            start_new = start-delta
        # определяем список кадров    
        for i in np.arange(start_new, end, 1 / saving_fps):
            s.append(i)
            number_of_violent.append(idx)
            if i < start:
                #  до начала нарушения флаг 0           
                flags.append(0)
            else:
                flags.append(1)
        end_old = end
    return (s, number_of_violent, flags)

In [9]:
def main(video_file, video_name, video_num, df, delta, save_dir, make_dir=False):
    """
    Функция выделяет в видео отрезки с нарушенями и за __ секунд до нарушения,
    создает кадры и сохраняет их в заданную папку.
    
    Args:
        video_file - путь до видео
        video_name - название видео
        video_num - номер видео
        df - данные по нарушениям для конкретного видео
        delta - сколько секунд отступаем до нарушения
        make_dir - сохраняем в одну папку или в разные
        save_dir - куда сохраняем
    """
    file_names = pd.DataFrame()
    
    # создаем папку для сохранения результатов
    if not os.path.isdir(save_dir):
        os.mkdir(save_dir)
        
    filename = save_dir
    
    if make_dir:
        filename, _ = os.path.splitext(video_name)
        filename = os.path.join(save_dir, filename+"_opencv")    
        # создаем индивидуальную папку для сохранения результатов
        if not os.path.isdir(filename):
            os.mkdir(filename)
            
    # читать видео файл    
    cap = cv2.VideoCapture(video_file)
    # получить FPS видео
    fps = cap.get(cv2.CAP_PROP_FPS)
    # если SAVING_FRAMES_PER_SECOND выше видео FPS, то установите его на FPS (как максимум)
    saving_frames_per_second = min(fps, SAVING_FRAMES_PER_SECOND)
    # получить список длительностей для сохранения
    saving_frames_durations, number_of_violent, flags = get_saving_frames_durations(cap, saving_frames_per_second, df, delta)
    # запускаем цикл
    count = 0
    save_frame_count = 0
    while True:
        is_read, frame = cap.read()
        if not is_read:
            # выйти из цикла, если нет фреймов для чтения
            break
        # получаем продолжительность, разделив количество кадров на FPS
        frame_duration = count / fps
        try:
            # получить самую раннюю продолжительность для сохранения
            closest_duration = saving_frames_durations[0]
        except IndexError:
            # список пуст, все кадры длительности сохранены
            break
        if frame_duration >= closest_duration:

            # если ближайшая длительность меньше или равна длительности кадра,
            # затем сохраняем фрейм
            frame_duration_formatted = format_timedelta(timedelta(seconds=frame_duration))           
            
            count_violation = number_of_violent[save_frame_count]
            flag = flags[save_frame_count]
            
            frame_name = f"video_{video_num}_frame_{frame_duration_formatted}_flag_{flag}_violation_{count_violation}.jpg"
#             print(frame_name)
            cv2.imwrite(os.path.join(filename, frame_name), frame) 
            file_df = pd.DataFrame({'video_name': video_name,
                                 'video_num': video_num,
                                'image_name': frame_name,
                                 'frame_duration': frame_duration_formatted,
                                 'target': flag}, index=[0])
            file_names = pd.concat([file_names, file_df], ignore_index=True)
    
#             file_names.append(frame_name)
            save_frame_count += 1
            # удалить точку продолжительности из списка, так как эта точка длительности уже сохранена
            try:
                saving_frames_durations.pop(0)
            except IndexError:
                pass
        # увеличить количество кадров
        count += 1
    return file_names

In [10]:
info_df = pd.read_excel('разметка.xlsx')

In [11]:
SAVING_FRAMES_PER_SECOND = 1 # кол-во кадров  в секунду
DELTA = 5 # отступаем до начала нарушения 5 сек

In [12]:
save_result_path = 'images'


image_result_info = pd.DataFrame()

for vi, f in tqdm(enumerate(info_df.videofile.unique()), position=0):
    temp = info_df[info_df.videofile==f]#.reset_index(drop=True)
    video_file = f'train/{f}'    
    file_names_df = main(video_file, f, vi, temp, DELTA, save_result_path)   
    image_result_info = pd.concat([image_result_info, file_names_df], ignore_index=True)

2it [00:08,  4.22s/it]


In [13]:
image_result_info.to_csv('image_result_info.csv')

In [14]:
image_result_info

,video_name,video_num,image_name,frame_duration,target
0,VID_20240311_154731.mp4,0,video_0_frame_0-00-00.00_flag_1_violation_0.jpg,0-00-00.00,1
1,VID_20240311_154731.mp4,0,video_0_frame_0-00-01.00_flag_1_violation_0.jpg,0-00-01.00,1
2,VID_20240311_154731.mp4,0,video_0_frame_0-00-02.00_flag_1_violation_0.jpg,0-00-02.00,1
3,VID_20240311_154731.mp4,0,video_0_frame_0-00-03.00_flag_1_violation_0.jpg,0-00-03.00,1
4,VID_20240311_154731.mp4,0,video_0_frame_0-00-04.00_flag_1_violation_0.jpg,0-00-04.00,1
5,VID_20240311_154731.mp4,0,video_0_frame_0-00-05.00_flag_1_violation_1.jpg,0-00-05.00,1
6,VID-20240301-WA0023.mp4,1,video_1_frame_0-02-14.00_flag_0_violation_2.jpg,0-02-14.00,0
7,VID-20240301-WA0023.mp4,1,video_1_frame_0-02-15.00_flag_0_violation_2.jpg,0-02-15.00,0
8,VID-20240301-WA0023.mp4,1,video_1_frame_0-02-16.00_flag_0_violation_2.jpg,0-02-16.00,0
9,VID-20240301-WA0023.mp4,1,video_1_frame_0-02-17.00_flag_0_violation_2.jpg,0-02-17.00,0
